In [1]:
# ============================================================
# stage0_7_offline_augmentation.py
# ------------------------------------------------------------
# stage0_6 기반 오프라인 증강 → stage0_7 데이터셋 생성기
# - 입력 : ../../data/meta_stage0_6_train_v6.csv
# - 출력 : ../../data/meta_stage0_7_train_v7.csv
# - 폴더 : ../../data/processed/stage0_6_train_v6 → stage0_7_train_v7
# - 컬럼 : filepath, group, basename
# ============================================================

import os
import cv2
import pandas as pd
from tqdm import tqdm
import albumentations as A

# ------------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------------
BASE = "../../data"
SRC_META = f"{BASE}/meta_stage0_6_train_v6.csv"
SRC_DIR  = f"{BASE}/processed/stage0_6_train_v6"
DST_DIR  = f"{BASE}/processed/stage0_7_train_v7"
DST_META = f"{BASE}/meta_stage0_7_train_v7.csv"

os.makedirs(DST_DIR, exist_ok=True)

# ------------------------------------------------------------
# 2️⃣ 증강기 정의 (Offline용)
# ------------------------------------------------------------
augment = A.Compose([
    A.Rotate(limit=180, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.5),
])

# ------------------------------------------------------------
# 3️⃣ 메타데이터 로드
# ------------------------------------------------------------
df = pd.read_csv(SRC_META)
print(f"✅ Loaded meta: {df.shape}, groups={df['group'].nunique()}")

# ------------------------------------------------------------
# 4️⃣ 증강 수행
# ------------------------------------------------------------
records = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting v6 → v7"):
    src_path = row["filepath"]
    group = row["group"]
    base = row["basename"]

    src_full = os.path.join(SRC_DIR, group, base)
    dst_group_dir = os.path.join(DST_DIR, group)
    os.makedirs(dst_group_dir, exist_ok=True)

    # 원본 이미지 로드
    img = cv2.imread(src_full)
    if img is None:
        print(f"⚠️ Cannot read: {src_full}")
        continue

    # (1) 원본 복사
    dst_path = os.path.join(dst_group_dir, base)
    if not os.path.exists(dst_path):
        cv2.imwrite(dst_path, img, [cv2.IMWRITE_JPEG_QUALITY, 95])
    records.append({
        "filepath": dst_path,
        "group": group,
        "basename": base
    })

    # (2) 증강 4종 생성
    for i in range(4):
        aug_img = augment(image=img)["image"]
        new_name = f"{os.path.splitext(base)[0]}_aug{i+1}.jpg"
        dst_aug_path = os.path.join(dst_group_dir, new_name)

        if not os.path.exists(dst_aug_path):
            cv2.imwrite(dst_aug_path, aug_img, [cv2.IMWRITE_JPEG_QUALITY, 95])

        records.append({
            "filepath": dst_aug_path,
            "group": group,
            "basename": new_name
        })

# ------------------------------------------------------------
# 5️⃣ 결과 CSV 저장
# ------------------------------------------------------------
out_df = pd.DataFrame(records)
out_df.to_csv(DST_META, index=False)
print(f"\n✅ Augmentation complete!")
print(f"📁 Saved to: {DST_META}")
print(f"📊 Total images: {len(out_df)} (≈ {len(df)} × 5)")


✅ Loaded meta: (1570, 3), groups=6


Augmenting v6 → v7: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1570/1570 [00:48<00:00, 32.31it/s]


✅ Augmentation complete!
📁 Saved to: ../../data/meta_stage0_7_train_v7.csv
📊 Total images: 7850 (≈ 1570 × 5)
